In [ ]:
!pip install emoji
!pip install langdetect

     |████████████████████████████████| 133kB 6.6MB/s 


In [ ]:
import pandas as pd
import requests
import json

In [ ]:
train = pd.read_csv("train.csv")
dev = pd.read_csv("dev.csv")
test = pd.read_csv("test.csv")

text = train.text.tolist()
text.extend(dev.text.tolist())
text.extend(test.text.tolist())

with open("vocab.txt","w") as f:
  for line in text:
    for word in line.split():
      f.write(word+"\n")


In [ ]:
with open("vocab.txt") as f:
  word_set = f.readlines()

In [ ]:
error_words = set()
transliteration_map = {}
visited_words = set()

for word in word_set:
  tword=word.strip("\n")
  if tword not in visited_words:
    visited_words.add(tword)
    if "#" in tword:
      tword =tword.replace("#","$$$")
    URL = f"https://www.google.com/inputtools/request?text={tword}&ime=transliteration_en_hi&num=5&cp=0&cs=0&ie=utf-8&oe=utf-8&app=jsapi&uv"

    PARAMS = {} 
    r = requests.get(url = URL, params = PARAMS)
    
    data = r.json()
    if data[0] != "SUCCESS":
      error_words.add(tword.replace("$$$","#"))
    else:
      try:
        transliteration_map[tword.replace("$$$","#")] = data[1][0][1][0].replace("$$$","#")
      except IndexError:
        error_words.add(tword.replace("$$$","#"))

In [ ]:
from emoji import UNICODE_EMOJI

def is_emoji(s):
    for i in s:
      
      if i in UNICODE_EMOJI['en'] or "🇮🇳" in s:
        return True
    return False
    
removed = []
for i in error_words:
  if is_emoji(i):
    removed.append(i)
    transliteration_map[i] = "इमोटिकॉन"

In [ ]:
for i in removed:
  error_words.remove(i)

In [ ]:
to_remove = ["¤¤¤¤¤¤","¥","°","³",'µ',
             "“","”",'‘','’',"•","…",
             "………….....",'₹','★','♡','♡♡♡',
             '♡♡♡♡♡','\U000fe334\U000fe334']

for i in to_remove:
    transliteration_map[i] = "इमोटिकॉन"

In [ ]:
for i in to_remove:
  error_words.remove(i)

In [ ]:
error_words

In [ ]:
transliteration_map["&_only"] ="&_ओनली"
transliteration_map["&theater"] = "&थिएटर"
transliteration_map["^exactly"] = "^एक्साक्ट्ली"
transliteration_map["atttttttttttttt"] = "अट्ट"
transliteration_map["bhaiiiiiiiiii"] = "भाई"
transliteration_map["bhaiiiiiiiiiii"] = "भाई"
transliteration_map["bhaiiiiiiiiiiii"] = "भाई"
transliteration_map["bhaiiiiiiiiiiiii"] = "भाई"
transliteration_map["bhaiiiiiiiiiiiiiii"] = "भाई"
transliteration_map["bhaiiiiiiiiiiiiiiiiiii"] = "भाई"
transliteration_map["biggggggggggg"] = "बिग"
transliteration_map['goooooooooooooo'] = 'जो'
transliteration_map['haaaaaaaaaaa'] = "है"
transliteration_map['loveeeeeeeeeeee'] = "लव"
transliteration_map['modijiiiiiiiiiiii'] = "मोदिजि"
transliteration_map['oooooooooo'] = "ओ"
transliteration_map['punjabbbbbbbbbbbbb'] = "पंजाब"
transliteration_map['salmaaaaaaaaaaan'] = 'सलमान'
transliteration_map['sooooooooooo'] = 'सो'
transliteration_map['sooooooooooooo'] = 'सो'
transliteration_map['soooooooooooooooooooooooooooooooooooooooooooooooo'] = 'सो'
transliteration_map['srrrrrrrrrrrr'] ='सर'
transliteration_map['sryyyyyyyyyyyy'] = 'श्री'
transliteration_map['thuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuu'] = 'ठु'
transliteration_map['trueeeeeeeeeeee'] = 'ट्रू'
transliteration_map['uhhhhhhhhhhhhh'] = 'यह'
transliteration_map['very^infinity'] = 'वैरी^इनफिनेटली'
transliteration_map['yrrrrrrrrrr'] = 'यर'
transliteration_map['khaaaaaaaaaaaaaan'] = 'खान'

In [ ]:
for i in transliteration_map:
  if i in error_words:
    error_words.remove(i)

In [ ]:
error_words

In [ ]:
with open("trans_map","w",encoding="utf-8") as f:
  json.dump(transliteration_map,f)

In [ ]:
def apply_trans_map(sentence):
  words = []
  for word in sentence.split():
    if word in transliteration_map:
      words.append(transliteration_map[word])
    else:
      words.append(word)
  return " ".join(words)



In [ ]:
train=pd.read_csv("train.csv")
train["text"]=train['text'].map(lambda sent: apply_trans_map(sent))
train.to_csv("hin_train.csv",index=False)

dev=pd.read_csv("dev.csv")
dev["text"]=dev['text'].map(lambda sent: apply_trans_map(sent))
dev.to_csv("hin_dev.csv",index=False)

test=pd.read_csv("test.csv")
test["text"]=test['text'].map(lambda sent: apply_trans_map(sent))
test.to_csv("hin_test.csv",index=False)


In [ ]:
import json
with open("hin_map.json","w") as f:
  json.dump(transliteration_map,f)

In [ ]:
text = test.text.tolist()
ans = []
with open("test.txt","w") as f:
  for i in text:
    f.write("\n".join(i.split())+"\n\n")

In [ ]:
!wc -l *.txt

  37906 etest.txt
  37906 test.txt
 146680 vocab.txt
 222492 total


In [ ]:
test=pd.read_csv("hin_test.csv")
text = test.text.tolist()
ans = []
with open("test5.txt","w") as f:
  for i in text:
    f.write("\n".join(i.split())+"\n\n")
!wc -l test5.txt

37906 test5.txt


In [ ]:
with open("test.txt") as f:
    text = []
    sent = []
    count = 0
    for line in f:
        if line != "\n":
            sent.append(line.lower().strip("\n"))
            
            count += 1
        else:
            if count == 0:
                continue
            else:
                text.append(" ".join(sent))
                sent = []
                

In [ ]:
test = pd.DataFrame({"text":text})
test.to_csv("hin_test.csv",index=False)